In [6]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

In [8]:
import openai
openai.organization = organization
openai.api_key = api_key
models = openai.Model.list()
models

<OpenAIObject list at 0x7f3992c0bdd0> JSON: {
  "data": [
    {
      "created": 1677532384,
      "id": "whisper-1",
      "object": "model",
      "owned_by": "openai-internal",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1683912666,
          "group": null,
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "whisper-1"
    },
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
      

In [16]:
gpt_model_name = "gpt-3.5-turbo"
[m for m in models.data if m.root == gpt_model_name][0]

<Model model id=gpt-3.5-turbo at 0x7f3993f48900> JSON: {
  "created": 1677610602,
  "id": "gpt-3.5-turbo",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1683852530,
      "group": null,
      "id": "modelperm-C34BeeteYvuLNoa893aJRcnp",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "gpt-3.5-turbo"
}

In [20]:
def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message

In [23]:
basic_completion("Write a haiku about being a dungeon master")

<OpenAIObject at 0x7f3993f53880> JSON: {
  "content": "Crafting tales with care,\nFaces lit by candle's flare,\nMy world comes alive.",
  "role": "assistant"
}

In [30]:
system_prompt = '''
You are an improvisational subsystem, part of a TTRPG game master assistant utility.

All of your responses must be a valid JSON object.
'''

def system_completion(user_prompt):
    call = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
    )
    return call.choices[0].message

world_prompt = '''
Create a standard fantasy world with the following structure:

name: ''
regions: ''
    name: ''
    description: ''
    pointsOfInterest:
        name: ''
        description: ''
    settlements:
        name: ''
        description: ''
        size: <VILLAGE|TOWN|CITY>
factions:
    name: ''
    description: ''
    locations:
        settlement: <reference>
        presence: <NONE|LIMITED|ESTABLISHED>

All settlements must be unique. All factions' locations must reference a settlement that exists.

There should be 3-4 regions, each with 1-2 points of interest and 3-6 settlements. 

There should be 2-3 major factions, and 5-10 minor factions. 
'''

def create_world():
    return system_completion(world_prompt)


In [31]:
world = create_world()
world

<OpenAIObject at 0x7f3993f61490> JSON: {
  "content": "{\n  \"name\": \"Fantasia\",\n  \"regions\": [\n    {\n      \"name\": \"The Great Forest\",\n      \"description\": \"A vast woodland of ancient trees and hidden dangers.\",\n      \"pointsOfInterest\": [\n        {\n          \"name\": \"The Sentinel Tree\",\n          \"description\": \"An enormous tree rumored to have a conscious spirit that watches over the forest.\"\n        },\n        {\n          \"name\": \"The Crystal Caves\",\n          \"description\": \"A series of underground caves filled with glittering crystals and dangerous creatures.\"\n        }\n      ],\n      \"settlements\": [\n        {\n          \"name\": \"Greenhaven\",\n          \"description\": \"A small elven village built within the branches of the trees.\",\n          \"size\": \"VILLAGE\"\n        },\n        {\n          \"name\": \"Oakheart\",\n          \"description\": \"A large human city on the edge of the forest, known for its skilled woodw

In [35]:
print(world['content'])

{
  "name": "Fantasia",
  "regions": [
    {
      "name": "The Great Forest",
      "description": "A vast woodland of ancient trees and hidden dangers.",
      "pointsOfInterest": [
        {
          "name": "The Sentinel Tree",
          "description": "An enormous tree rumored to have a conscious spirit that watches over the forest."
        },
        {
          "name": "The Crystal Caves",
          "description": "A series of underground caves filled with glittering crystals and dangerous creatures."
        }
      ],
      "settlements": [
        {
          "name": "Greenhaven",
          "description": "A small elven village built within the branches of the trees.",
          "size": "VILLAGE"
        },
        {
          "name": "Oakheart",
          "description": "A large human city on the edge of the forest, known for its skilled woodworkers.",
          "size": "CITY"
        },
        {
          "name": "Wildwood",
          "description": "A rough-and-tumble h

In [40]:
from jsoncomment import JsonComment
json = JsonComment()

world_parsed = json.loads(world['content'])
world_parsed

{'name': 'Fantasia',
 'regions': [{'name': 'The Great Forest',
   'description': 'A vast woodland of ancient trees and hidden dangers.',
   'pointsOfInterest': [{'name': 'The Sentinel Tree',
     'description': 'An enormous tree rumored to have a conscious spirit that watches over the forest.'},
    {'name': 'The Crystal Caves',
     'description': 'A series of underground caves filled with glittering crystals and dangerous creatures.'}],
   'settlements': [{'name': 'Greenhaven',
     'description': 'A small elven village built within the branches of the trees.',
     'size': 'VILLAGE'},
    {'name': 'Oakheart',
     'description': 'A large human city on the edge of the forest, known for its skilled woodworkers.',
     'size': 'CITY'},
    {'name': 'Wildwood',
     'description': 'A rough-and-tumble halfling settlement in the heart of the forest, known for its skilled hunters.',
     'size': 'TOWN'}]},
  {'name': 'The Iron Mountains',
   'description': 'A range of towering, jagged peak

In [41]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(world_parsed)

{'factions': [{'description': 'A powerful army made up of soldiers and '
                              'mercenaries from all over the continent.',
               'locations': [{'presence': 'ESTABLISHED',
                              'settlement': 'Ironhold'},
                             {'presence': 'LIMITED',
                              'settlement': "Skullcrusher's Keep"}],
               'name': 'The Iron Legion'},
              {'description': 'A secretive group of druids and rangers who '
                              'seek to protect the natural world from harm.',
               'locations': [{'presence': 'ESTABLISHED',
                              'settlement': 'Greenhaven'},
                             {'presence': 'LIMITED', 'settlement': 'Wildwood'}],
               'name': 'The Emerald Enclave'},
              {'description': 'A powerful network of thieves and assassins who '
                              'control much of the criminal underworld.',
               'loca